In [2]:
import pandas as pd


In [3]:
df=pd.read_csv("C:/machihe learning/Projects/Customer segmentaion and retention analysis/dataset/Online_Retail.csv", encoding='latin-1')

In [4]:
df.columns

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country'],
      dtype='object')

In [5]:
df.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


In [7]:
df=df.dropna(subset=['CustomerID'])

In [8]:
df.isnull().sum()

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

In [9]:
df['InvoiceNo']

0         536365
1         536365
2         536365
3         536365
4         536365
           ...  
541904    581587
541905    581587
541906    581587
541907    581587
541908    581587
Name: InvoiceNo, Length: 406829, dtype: object

In [10]:
# Returns the total number of rows starting with 'C'
count = df['InvoiceNo'].str.startswith('C', na=False).sum()
print(f"Number of invoices starting with 'C': {count}")


Number of invoices starting with 'C': 8905


In [11]:
df = df[~df['InvoiceNo'].str.startswith('C', na=False)]

In [14]:
neg_count = (df['UnitPrice'] < 0).sum()
print(f"Total negative rows: {neg_count}")


Total negative rows: 0


In [1]:
import pandas as pd
import numpy as np

In [7]:
raw_data=pd.read_csv("C:\machihe learning\Projects\AI shoping agent\dataset\Online_Retail.csv", encoding='latin-1')

In [8]:
# --- STEP 2: THE CLEANING FUNCTION ---
def clean_retail_data(df):
    print("Starting data cleaning...")
    
    # Drop missing CustomerIDs
    df = df.dropna(subset=['CustomerID'])
    
    # Remove cancellations and non-positive values
    df = df[~df['InvoiceNo'].astype(str).str.startswith('C')]
    df = df[(df['Quantity'] > 0) & (df['UnitPrice'] > 0)]
    
    # Format types
    df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
    df['CustomerID'] = df['CustomerID'].astype(int)
    
    # Create TotalSum
    df['TotalSum'] = df['Quantity'] * df['UnitPrice']
    
    # Remove duplicates
    df = df.drop_duplicates()
    
    print(f"Cleaning complete. New dataset shape: {df.shape}")
    return df

In [9]:
# --- STEP 3: RUN AND CHECK ---
df_clean = clean_retail_data(raw_data)

# Check the baseline numbers for Phase 2
print("\n--- BASELINE CHECKS ---")
print(f"Date Range: {df_clean['InvoiceDate'].min()} to {df_clean['InvoiceDate'].max()}")
print(f"Unique Customers: {df_clean['CustomerID'].nunique()}")
print(f"Total Revenue: ${df_clean['TotalSum'].sum():,.2f}")

Starting data cleaning...


C:\Users\hp\AppData\Local\Temp\ipykernel_22956\4218397572.py:13: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])


Cleaning complete. New dataset shape: (392692, 9)

--- BASELINE CHECKS ---
Date Range: 2010-12-01 08:26:00 to 2011-12-09 12:50:00
Unique Customers: 4338
Total Revenue: $8,887,208.89


# RFM Calculation

In [10]:
import datetime as dt

In [11]:
# 1. Set the snapshot date (one day after the last purchase)
snapshot_date = df_clean['InvoiceDate'].max() + dt.timedelta(days=1)


In [12]:
# 2. Aggregate data by CustomerID
rfm = df_clean.groupby('CustomerID').agg({
    'InvoiceDate': lambda x: (snapshot_date - x.max()).days, # Recency
    'InvoiceNo': 'count',                                   # Frequency
    'TotalSum': 'sum'                                       # Monetary
})

In [13]:
# 3. Rename columns for clarity
rfm.rename(columns={
    'InvoiceDate': 'Recency',
    'InvoiceNo': 'Frequency',
    'TotalSum': 'Monetary'
}, inplace=True)

print("RFM Table Created Successfully!")
print(rfm.head())

RFM Table Created Successfully!
            Recency  Frequency  Monetary
CustomerID                              
12346           326          1  77183.60
12347             2        182   4310.00
12348            75         31   1797.24
12349            19         73   1757.55
12350           310         17    334.40


In [14]:
rfm.describe()

,Recency,Frequency,Monetary
count,4338.000000,4338.000000,4338.000000
mean,92.536422,90.523744,2048.688081
std,100.014169,225.506968,8985.230220
min,1.000000,1.000000,3.750000
25%,18.000000,17.000000,306.482500
50%,51.000000,41.000000,668.570000
75%,142.000000,98.000000,1660.597500
max,374.000000,7676.000000,280206.020000


In [ ]:
import pandas as pd
import datetime as dt

# Load data (Ensure the filename matches yours)
df = pd.read_csv('C:\machihe learning\Projects\AI shoping agent\dataset\Online_Retail.csv', encoding="ISO-8859-1")

# 1. Professional Cleaning
df = df.dropna(subset=['CustomerID'])
df = df[~df['InvoiceNo'].astype(str).str.startswith('C')]
df = df[(df['Quantity'] > 0) & (df['UnitPrice'] > 0)]
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
df['TotalSum'] = df['Quantity'] * df['UnitPrice']

# 2. RFM Calculation
snapshot_date = df['InvoiceDate'].max() + dt.timedelta(days=1)
rfm = df.groupby('CustomerID').agg({
    'InvoiceDate': lambda x: (snapshot_date - x.max()).days,
    'InvoiceNo': 'nunique', # Count unique orders, not just rows
    'TotalSum': 'sum'
}).rename(columns={'InvoiceDate': 'Recency', 'InvoiceNo': 'Frequency', 'TotalSum': 'Monetary'})

print("Consultant Foundation Ready!")
print(f"Total Clients Analyzed: {len(rfm)}")
